# 04. Semantic Kernel

In this lab, we will introduce **Semantic Kernel**. Like **Langchain**, it provides a framework for working with AI models and supports applications written in Python. Semantic Kernel also adds support for .NET and Java applications.

As with the other labs, we'll start by reading in values from the `.env` file.

___
**NOTE**: This lab uses **.NET** so be sure to select .NET and not Python when starting to use this notebook.
___

In [ ]:
#r "nuget: dotenv.net, 3.1.2"

using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

// Load values into variables and strip quotes
var model = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azureEndpoint = envVars["OPENAI_API_BASE"].Replace("\"", "");
var apiKey = envVars["OPENAI_API_KEY"].Replace("\"", "");


To start working with Semantic Kernel, we'll need to load its nuget package.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"

Semantic Kernel works by creating an instance of the Kernel and then adding in various plugins to perform different functions. Those addins or functions can then be called individually or chained together to perform more complex tasks.

We use the standard .NET `builder` pattern to initialise the kernel. Notice that we pass in the details of the completion model that we're going to use, the Azure OpenAI API endpoint URL and the API key.

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();
builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);

IKernel kernel = builder.Build();

Let's create a Semantic Function to perform a simple query. We start by creating a prompt template. Notice that we use a placeholder - `{{$input}}` - to represent a value that we will pass in later.

In [ ]:
string prompt = "What interesting things can I make with a {{$input}}?";

Next we create a Semantic Function using our prompt and register it with the kernel.

In [ ]:
var whatCanIMakeFunction = kernel.CreateSemanticFunction(prompt);

Finally, we invoke the function with a value passed in to replace our `{{$input}}` variable.

In [ ]:
var response = await whatCanIMakeFunction.InvokeAsync("raspberry pi");

Console.WriteLine(response);

The result is a long list of ideas for things we could make with a Raspberry Pi. 

Now, let's look at chaining functions together. We can configure Semantic Kernel to perform multiple tasks, using the output of the first task as the `$input` for the second and so on.

Let's improve the above query by getting a summary of the options that were generated. First, we'll create a new prompt that will generate a summary of some text.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

string summaryPrompt = "Summarize the following text: {{$input}}";
var summaryFunction = kernel.CreateSemanticFunction(summaryPrompt);

With our new semantic function created, let's now run it together with the original prompt to generate ideas. The following code will initiate the `whatCanIMakeFunction` with the value `raspberry pi` and then pass the output from that function to the `summaryFunction`.

In [ ]:
var context = await kernel.RunAsync(
    new ContextVariables("raspberry pi"),
    whatCanIMakeFunction,
    summaryFunction);

Console.WriteLine(context);

## Summary

Semantic Kernel is another example of an AI orchestrator. Like other orchestrators, it can be used to simplify the process of creating complex AI applications.

## Next Section

📣 [AI Orchestration](../../03-orchestration/README.md)